In [3]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from util.logUtil import Logger

In [4]:
# 日志
# 初始化日志
logger = Logger(log_dir='../log', log_name='train').get_logger()
logger.info("=" * 50)
logger.info("2025-8-10 算法练习")

train - INFO - ==================================================
train - INFO - 2025-8-10 算法练习


# 2322. 从树中删除边的最小分数
![2322](pic/2322.png)
nums = [1,5,5,4,11]
edges = [[0,1],[1,2],[1,3],[3,4]]

删除两条边会将树分成三个部分，这三个部分的异或值可以通过子树异或和计算。<br>
代码中dfs 计算以 x 为根的子树的异或和 son，dfs2 则处理第二条边的分割。<br>


In [8]:
class Solution:
    def calc(self, part1, part2, part3):
        return max(part1, part2, part3) - min(part1, part2, part3)
    def minimumScore(self, nums, edges):
        n = len(nums)
        e = [[] for _ in range(n)]
        for u, v in edges:
            e[u].append(v)
            e[v].append(u)
        # 建一个邻接表
        total = 0
        for x in nums:
            total ^= x
        res = float('inf')
       # 每次递归调用会生成一个独立的son变量，用来保存当前节点为根的子树异或和
        def dfs2(x, f, oth, anc) :
            son = nums[x]
            for y in e[x]:
                if y == f:
                    continue
                son ^= dfs2(y, x, oth, anc)
            if f == anc:
                return son
            nonlocal res
            res = min(res, self.calc(oth, son, total ^ oth ^ son))
            return son
        
        def dfs(x, f) :
            son = nums[x]
            for y in e[x]:
                if y == f:
                    continue
                son ^= dfs(y, x)
            for y in e[x]:
                if y == f:
                    dfs2(y, x, son, x)
            return son

        dfs(0, -1)
        return res
nums = [1,5,5,4,11]
edges = [[0,1],[1,2],[1,3],[3,4]]
Solution().minimumScore(nums, edges)

9

In [13]:
class Solution:
    def calc(self, part1, part2, part3):
        """
        计算三个部分异或值的分数（最大值与最小值的差）
        :param part1: 第一部分的异或和
        :param part2: 第二部分的异或和
        :param part3: 第三部分的异或和
        :return: 三个值的最大值与最小值之差
        """
        return max(part1, part2, part3) - min(part1, part2, part3)
    
    def minimumScore(self, nums, edges):
        """
        计算删除两条边后三个连通分量异或值的最小分数
        :param nums: List[int] 节点值数组，nums[i]表示节点i的值
        :param edges: List[List[int]] 边列表，每条边连接两个节点
        :return: int 最小可能的分数
        """
        n = len(nums)
        # 构建邻接表表示的无向树
        e = [[] for _ in range(n)]
        for u, v in edges:
            e[u].append(v)
            e[v].append(u)
        
        # 计算所有节点值的异或总和
        total = 0
        for x in nums:
            total ^= x
        
        # 初始化结果为无穷大
        res = float('inf')
        
        def dfs2(x, f, oth, anc):
            """
            深度优先搜索，处理第二条被删除边的情况
            :param x: 当前节点
            :param f: 父节点
            :param oth: 第一条边分割后其中一个部分的异或和
            :param anc: 祖先节点（第一条边的上半部分连接点）
            :return: 以x为根的子树异或和
            """
            # 初始化当前子树异或和为当前节点值
            son = nums[x]
            # 遍历当前节点的所有邻居（子节点）
            for y in e[x]:
                # 跳过父节点，避免向上遍历
                if y == f:
                    continue
                # 递归计算子树异或和，并异或到当前子树异或和中
                son ^= dfs2(y, x, oth, anc)
            
            # 如果父节点是祖先节点，说明这是第一条边分割的部分，直接返回
            # 这意味着我们不计算这种情况下的分数，因为这不是一个有效的分割
            if f == anc:
                return son
            
            # 更新最小分数：
            # 三个部分的异或和分别是：
            # 1. oth: 第一条边分割出的第一部分
            # 2. son: 第二条边分割出的第二部分（以x为根的子树）
            # 3. total ^ oth ^ son: 剩余的第三部分
            nonlocal res
            res = min(res, self.calc(oth, son, total ^ oth ^ son))
            
            return son
        
        def dfs(x, f):
            """
            深度优先搜索，处理第一条被删除边的情况
            :param x: 当前节点
            :param f: 父节点
            :return: 以x为根的子树异或和
            """
            # 初始化当前子树异或和为当前节点值
            son = nums[x]
            
            # 第一次遍历：计算以x为根的子树的异或和
            for y in e[x]:
                # 跳过父节点，避免向上遍历
                if y == f:
                    continue
                # 递归计算子树异或和，并异或到当前子树异或和中
                son ^= dfs(y, x)
            # 第二次遍历：尝试选择第二条边
            # 注意：这里的逻辑是只对父节点调用dfs2
            # 这实际上是在考虑删除从当前节点x到其父节点f的边作为第一条边
            for y in e[x]:
                if y == f:
                    # 调用dfs2处理第二条边的分割
                    # 参数含义：
                    # y: 父节点（作为dfs2的起始节点）
                    # x: 当前节点（作为dfs2的父节点，即第一条边的下端点）
                    # son: 以x为根的子树异或和（第一条边分割出的一部分）
                    # x: 祖先节点（用于在dfs2中判断是否为第一条边的分割）
                    dfs2(y, x, son, x)
            return son

        # 从节点0开始DFS，-1表示节点0没有父节点
        dfs(0, -1)
        
        return res

# 测试用例
nums = [1,5,5,4,11]
edges = [[0,1],[1,2],[1,3],[3,4]]
Solution().minimumScore(nums, edges)


9